**Upload batch file**
Once your input file is prepared, you first need to upload the file to then be able to kick off a batch job. File upload can be done both programmatically or via the Studio. This example uses environment variables in place of the key and endpoint values. If you're unfamiliar with using environment variables with Python refer to one of our quickstarts where the process of setting up the environment variables in explained step-by-step.

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(".env")  # make sure to have the .env file in the root directory of the project
  
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("test2-base64-encoded-image.jsonl", "rb"), 
  purpose="batch"
)

print(file.model_dump_json(indent=2))


{
  "id": "file-b4ecf74cd14c40ffb1f0cc5680a5eebc",
  "bytes": 1646,
  "created_at": 1722921938,
  "filename": "test2-base64-encoded-image.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "pending",
  "status_details": null
}


copy the "id" value from the json output from previous step
define the variable file_id and assign the value to it

In [2]:
file_id ="file-b4ecf74cd14c40ffb1f0cc5680a5eebc"

**Track file upload status**
Depending on the size of your upload file it might take some time before it's fully uploaded and processed. To check on your file upload status run:

In [3]:
# Wait until the uploaded file is in processed state
import time
import datetime 
file_id = file_id
status = "pending"
while status != "processed":
    time.sleep(15)
    file_response = client.files.retrieve(file_id)
    status = file_response.status
    print(f"{datetime.datetime.now()} File Id: {file_id}, Status: {status}")

2024-08-05 22:26:20.477461 File Id: file-b4ecf74cd14c40ffb1f0cc5680a5eebc, Status: processed



**Create batch job**
Once your file has uploaded successfully by reaching a status of processed you can submit the file for batch processing.

In [4]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
)

# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_2608eba1-99ff-4799-9f67-0c9395793a86",
  "completion_window": "24h",
  "created_at": 1722922012,
  "endpoint": "/chat/completions",
  "input_file_id": "file-b4ecf74cd14c40ffb1f0cc5680a5eebc",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1723008412,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


copy the "id" value from the json output from previous step
define the variable batch_id and assign the value to it

In [5]:
batch_id="batch_2608eba1-99ff-4799-9f67-0c9395793a86"

 **Note**

Currently the completion window must be set to 24h. If you set any other value than 24h your job will fail. Jobs taking longer than 24 hours will continue to execute until cancelled.


**Track batch job progress**
Once you have created batch job successfully you can monitor its progress either in the Studio or programatically. When checking batch job progress we recommend waiting at least 60 seconds in between each status call.

In [6]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

2024-08-05 22:28:38.041504 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: validating
2024-08-05 22:29:38.966142 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: in_progress
2024-08-05 22:30:39.824192 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: in_progress
2024-08-05 22:31:40.373427 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: finalizing
2024-08-05 22:32:40.992260 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: finalizing
2024-08-05 22:33:41.760701 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: finalizing
2024-08-05 22:34:42.353162 Batch Id: batch_2608eba1-99ff-4799-9f67-0c9395793a86,  Status: completed


The following status values are possible:
| Status       | Description                                                                 |
|--------------|-----------------------------------------------------------------------------|
| validating   | The input file is being validated before the batch processing can begin.    |
| failed       | The input file has failed the validation process.                           |
| in_progress  | The input file was successfully validated and the batch is currently running.|
| finalizing   | The batch has completed and the results are being prepared.                 |
| completed    | The batch has been completed and the results are ready.                     |
| expired      | The batch wasn't able to be completed within the 24-hour time window.       |
| cancelling   | The batch is being cancelled (This may take up to 10 minutes to go into effect.)|
| cancelled    | The batch was cancelled.                                                    |


To examine the job status details you can run:

In [7]:
print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_2608eba1-99ff-4799-9f67-0c9395793a86",
  "completion_window": "24h",
  "created_at": 1722922012,
  "endpoint": "/chat/completions",
  "input_file_id": "file-b4ecf74cd14c40ffb1f0cc5680a5eebc",
  "object": "batch",
  "status": "completed",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": 1722922455,
  "error_file_id": "file-5b6f18c1-ec5b-4fff-aaea-7de86409bee9",
  "errors": null,
  "expired_at": null,
  "expires_at": 1723008412,
  "failed_at": null,
  "finalizing_at": 1722922299,
  "in_progress_at": 1722922149,
  "metadata": null,
  "output_file_id": "file-85bae9c5-3906-4040-9ad9-29134f36330b",
  "request_counts": {
    "completed": 1,
    "failed": 0,
    "total": 1
  }
}


Observe that there's both error_file_id and a separate output_file_id. Use the error_file_id to assist in debugging any issues that occur with your batch job.

**Retrieve batch job output file**

In [11]:
import json

response = client.files.content(batch_response.output_file_id)
raw_responses = response.text.strip().split('\n')  

for raw_response in raw_responses:  
    json_response = json.loads(raw_response)  
    formatted_json = json.dumps(json_response, indent=2)  
    print(formatted_json)

{
  "custom_id": "new-batch-request-1",
  "response": {
    "body": {
      "choices": [
        {
          "content_filter_results": {
            "hate": {
              "filtered": false,
              "severity": "safe"
            },
            "self_harm": {
              "filtered": false,
              "severity": "safe"
            },
            "sexual": {
              "filtered": false,
              "severity": "safe"
            },
            "violence": {
              "filtered": false,
              "severity": "safe"
            }
          },
          "finish_reason": "stop",
          "index": 0,
          "logprobs": null,
          "message": {
            "content": "The picture is an emoji with a yellow face, heart-shaped eyes, and an open smile. The emoji conveys feelings of love, adoration, or being enamored with something.",
            "role": "assistant"
          }
        }
      ],
      "created": 1722922204,
      "id": "chatcmpl-9t73kXoNzy1BsFIcw


**Additional batch commands**
Cancel batch
Cancels an in-progress batch. The batch will be in status cancelling for up to 10 minutes, before changing to cancelled, where it will have partial results (if any) available in the output file.

In [ ]:
client.batches.cancel("batch_abc123") # set to your batch_id for the job you want to cancel

**List batch**
List all batch jobs for a particular Azure OpenAI resource.

In [12]:
client.batches.list()

SyncCursorPage[Batch](data=None, value=[{'cancelled_at': None, 'cancelling_at': None, 'completed_at': 1722922455, 'completion_window': '24h', 'created_at': 1722922012, 'error_file_id': 'file-5b6f18c1-ec5b-4fff-aaea-7de86409bee9', 'expired_at': None, 'expires_at': 1723008412, 'failed_at': None, 'finalizing_at': 1722922299, 'id': 'batch_2608eba1-99ff-4799-9f67-0c9395793a86', 'in_progress_at': 1722922149, 'input_file_id': 'file-b4ecf74cd14c40ffb1f0cc5680a5eebc', 'errors': None, 'metadata': None, 'object': 'batch', 'output_file_id': 'file-85bae9c5-3906-4040-9ad9-29134f36330b', 'request_counts': {'total': 1, 'completed': 1, 'failed': 0}, 'status': 'completed', 'endpoint': '/chat/completions'}, {'cancelled_at': None, 'cancelling_at': None, 'completed_at': None, 'completion_window': '24h', 'created_at': 1722921462, 'error_file_id': None, 'expired_at': None, 'expires_at': 1723007862, 'failed_at': 1722921553, 'finalizing_at': None, 'id': 'batch_e7cfaed2-f882-431a-bd8b-f3fffd87f822', 'in_progres

Global batch limits
| Limit Name            | Limit Value |
|-----------------------|-------------|
| Max files per resource| 500         |
| Max input file size   | 200 MB      |
| Max requests per file | 100,000     |

| Model         | Enterprise agreement | Default | Monthly credit card based subscriptions | MSDN subscriptions | Azure for Students, Free Trials |
|---------------|----------------------|---------|-----------------------------------------|--------------------|----------------------------------|
| gpt-4o        | 5 B                  | 50 M    | 1.35 M                                  | 90 K               | N/A                              |
| gpt-4-turbo   | 300 M                | 40 M    | 1.35 M                                  | 90 K               | N/A                              |
| gpt-4         | 150 M                | 5 M     | 200 K                                   | 100 K              | N/A                              |
| gpt-35-turbo  | 10 B                 | 100 M   | 5 M                                     | 2 M                | 50 K                             |
